# Config File Structure

In [1]:
import tensorflow as tf
import json

The hyper-parameter dictionary used for model tuning, fitting and data structue has the following form:
```python3
hyper = {
    "info":{ 
        # General information for training run
        "kgcnn_version": "1.1.0", # Version 
        "postfix": "" # postfix for output folder
    },
    "model": { 
        # Model specific parameter, see kgcnn.literature
    },
    "data": { 
        # Dataset specific parameter
    },
    "training": {
        "fit": { 
            # keras fit arguments serialized
        },
        "compile": { 
            # Keras compile arguments serialized
        },
        "Kfold": {
            # kwargs unpacked in scikit-learn Kfold class.  
        }
    }
}
```
The following sections explain each block.


In [2]:
hyper = {}

## Model

The model parameters can be reviewed from the default values in ``kgcnn.literature``. Mostly model input and output has to be matched depending on the data representation. That is type of input and its shape. An input-type checker will be added in the future to adapt the model input automatically, but as for now the input shape has to be chosen properly. In ``inputs`` a list of kwargs must be given, which are each unpacked in the corresponding ``tf.keras.layers.Input``. The order matters and is model dependent.

Moreover, naming of the model input is used to link the tensor properties of the dataset with the model input. 

In [3]:
hyper.update({
    "model":{
        "inputs": [
            {"shape": [None, 100], "name": "node_attributes", "dtype": "float32", "ragged": True},
            {"shape": [None, 2], "name": "edge_indices", "dtype": "int64", "ragged": True}],
        # More model specific kwargs, like:
        # "depth": 5,
        "output_mlp": {"use_bias": [True, True, False], "units": [140, 70, 70],
                       "activation": ["relu", "relu", "softmax"]}}
})

Here, the training script will provide ``dataset.edge_indices`` of shape `(batch, None, 2)` and ``dataset.node_attributes`` of shape `(batch, None, 100)`. Note that the shape must match the actual shape in dataset.

For output, most models simply have a MLP at the output and the activation as well as the final output dimension can be chosen by setting the kwargs ``output_mlp`` (unpacked in MLP) for last layer in ``units`` and ``activation``. The number in units must macht the labels or classes of the target. This is moslty ``dataset.graph_labels``, but depends on dataset and classification task, either graph or node classification.

## Data

The kwargs for the dataset are not fully identical and vary a little depending on the datset. However, the most common are listed below.

Importantly, if the kwargs for example ``set_range`` appears in the data kwargs, this will mean that ``dataset.set_range(**kwargs)`` will be executed before training.

In [4]:
hyper.update({
    "data":{
        "dataset": {},  # Used for constructor, leave out for pre-defined datasets
        "prepare_data": {},  # Used for cache and pre-compute data, leave out for pre-defined datasets
        "read_in_memory": {},  # Used for reading into memory, leave out for pre-defined datasets
        "set_attributes": {},  # If set, modify attributes for dataset.
        "set_edge_indices_reverse": {},  # If set, compute edge pair map indices like (0,1) to (1,0)
        "set_range": {},  # If set, define range edges based on geoemtric distance and store at dataset.range_indices
        "set_angle": {},  # If set, define anlge indices based on dataset.range_indices and store at dataset.angle_indices
    }
})

## Training

The kwargs for training simply sets arguments for ``model.compile(**kwargs)`` and ``model.fit(**kwargs)`` that matches keras arguments as well as for the k-fold split from scikit-learn. For now the kwargs are expected to be fully serialized as shown below.

In [5]:
hyper.update({
    "training":{
        "KFold": {"n_splits": 10, "random_state": None, 
                  "shuffle": True}, # Info on K-fold split used as sklearn.model_selection.KFold(**kwargs)
        "execute_folds": 10,  # For some training script running all folds would be too expensive for testing.
        
        # Keras model compile and fit
        "fit": {
            "batch_size": 32, "epochs": 800, "verose": 2, 
            "callbacks": []  # Note: tf.keras.callbacks can not be serialized at the moment
        },
        "compile": {
            "loss": "categorical_crossentropy",
            "optimizer": tf.keras.optimizers.Adam(learning_rate=0.001).get_config()
        }
    }
})

## Info

Some general information on the training, such the used kgcnn version or a postfix for the output files.

In [6]:
hyper.update({
    "info":{ # Generla information
        "postfix": "_v1", # Appends _v1 to output folder
        "postfix_file": "_run2", # Appends _run2 to log files
        "kgcnn_version": "1.1.0"    
    }
})

# Summary

Final hyper dictionary which can be fed to training script:

In [7]:
hyper

{'model': {'inputs': [{'shape': [None, 100],
    'name': 'node_attributes',
    'dtype': 'float32',
    'ragged': True},
   {'shape': [None, 2],
    'name': 'edge_indices',
    'dtype': 'int64',
    'ragged': True}],
  'output_mlp': {'use_bias': [True, True, False],
   'units': [140, 70, 70],
   'activation': ['relu', 'relu', 'softmax']}},
 'data': {'dataset': {},
  'prepare_data': {},
  'read_in_memory': {},
  'set_attributes': {},
  'set_edge_indices_reverse': {},
  'set_range': {},
  'set_angle': {}},
 'training': {'KFold': {'n_splits': 10, 'random_state': None, 'shuffle': True},
  'execute_folds': 10,
  'fit': {'batch_size': 32, 'epochs': 800, 'verose': 2, 'callbacks': []},
  'compile': {'loss': 'categorical_crossentropy',
   'optimizer': {'name': 'Adam',
    'learning_rate': 0.001,
    'decay': 0.0,
    'beta_1': 0.9,
    'beta_2': 0.999,
    'epsilon': 1e-07,
    'amsgrad': False}}},
 'info': {'postfix': '_v1', 'postfix_file': '_run2', 'kgcnn_version': '1.1.0'}}